<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture
!pip install langchain>=0.1.17 langchain_groq>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community

In [2]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from google.colab import userdata


# Load OpenAI's LLMs with LangChain
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
openai_llm = ChatGroq(model_name="llama-3.3-70b-versatile")

In [3]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [31]:
from langchain.agents import load_tools, Tool
from langchain_community.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [32]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [33]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
To find the current price of a MacBook Pro in USD and then convert it to EUR, I need to first find the current price of a MacBook Pro in USD. 

Action: duckduck
Action Input: current price of MacBook Pro in USD

DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

In [21]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain

In [25]:
%%capture
!pip install tavily-python

In [28]:
from langchain_community.tools import TavilySearchResults

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# You can create the tool to pass to an agent
search = TavilySearchResults()
tavily_tool = Tool(
    name="tavily_search",
    description="A search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools([], llm=llm)
tools.append(tavily_tool)

# Re-initialize the agent with the new tools
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [29]:
# run the agent
agent.run("Who are Dr. Raj Dandekar's co-founders? What is their current age raised to the 1 power?")



> Entering new AgentExecutor chain...
Thought: To find the co-founders of Dr. Raj Dandekar and their current age raised to the power of 1, I first need to identify who Dr. Raj Dandekar's co-founders are. 

Action: tavily_search
Action Input: Dr. Raj Dandekar co-founders
Observation: [{'title': 'About Us – Vizuara AI Labs', 'url': 'https://vizuara.ai/about-us/', 'content': 'Dr. Raj Abhijit Dandekar. MIT PhD, IIT Madras BTech. Co-founder, Vizuara AI Labs ; Dr. Rajat Dandekar\u200b. Purdue PhD, IIT Madras BTech Co-founder, Vizuara AI Labs', 'score': 0.81595254}, {'title': 'Elets Education - Facebook', 'url': 'https://www.facebook.com/eletseducation/posts/co-founders-of-vizuara-raj-dandekar-rajat-dandekar-and-sreedath-panat-share-abou/5058055724228072/', 'content': 'Co-founders of Vizuara, Raj Dandekar, Rajat Dandekar and Sreedath Panat share about their brand at an industry presentation at 22nd Elets', 'score': 0.79883134}, {'title': 'In praise of "Building LLMs from scratch" by Sebasti

'The co-founders of Dr. Raj Dandekar are Rajat Dandekar and Sreedath Panat. Unfortunately, their current age raised to the power of 1 cannot be calculated as their birthdates are not available.'

Now you can try running the agent again with the Tavily search tool.

In [30]:
# run the agent
agent.run("Who is Sunkaraboina Praveen Kumar?")



> Entering new AgentExecutor chain...
To find out who Sunkaraboina Praveen Kumar is, I first need to consider the available tools and determine which one is most suitable for this task. Given the tools provided, the tavily_search function appears to be the most appropriate for general queries, including searching for information about individuals.

Action: tavily_search
Action Input: 'Sunkaraboina Praveen Kumar'
Observation: [{'title': 'SunkaraboinaPraveenKumar - GitHub', 'url': 'https://github.com/SunkaraboinaPraveenKumar', 'content': '.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NTA5MTA3MjksIm5iZiI6MTc1MDkxMDQyOSwicGF0aCI6Ii8xNjAwOTI5MzAvNDU3NTY1NjkyLWRhYjBkOWVjLTIyNDQtNGQ4Ni1iMjk0LWIxNTI2ZWMwYWZhOS5wbmc_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ3JlZGVudGlhbD1BS0lBVkNPRFlMU0E1M1BRSzRaQSUyRjIwMjUwNjI2JTJGdXMtZWFzdC0xJTJGczMlMkZhd3M0X3JlcXVlc3QmWC1BbXotRGF0ZT0yMDI1MDYyNlQwNDAwMjlaJlgtQW16LUV4cGlyZXM9MzAwJlgtQW16LVNpZ2

'Sunkaraboina Praveen Kumar is a student or professional in the field of web development and machine learning, with experience in various programming languages and tools.'